# 1. Speech Recognition 설치

In [ ]:
import speech_recognition as sr

In [2]:
sr.__version__ #버전 확인

'3.8.1'

speech_recognition의 Recognizer 객체 생성하여 음성인식 기능을 사용함

In [3]:
r = sr.Recognizer()

# 2. Audio files 사용하기

Harvard.wav라는 파일을 열고 음성을 확인함

context manager이 파일을 열고 데이터를 읽은 후 source라는 AudioFile 인스턴스에 저장함

record() 메소드로 파일의 모든 데이터를 record함

In [5]:
harvard = sr.AudioFile('harvard.wav')
with harvard as source: 
    audio = r.record(source) 

In [6]:
type(audio)

speech_recognition.AudioData

recognize_google()을 사용해 음성 파일을 텍스트로 변환함

In [7]:
r.recognize_google(audio)

'the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle taste fine with ham tacos al Pastore are my favorite a zestful food is be hot cross bun'

* Offset and duration을 사용해 특정 부분을 segmentation하기

1. Duration을 사용하여 음성 파일의 첫 4초만 인식하여 transcribe함

In [8]:
with harvard as source:
    audio = r.record(source, duration=4)
    
r.recognize_google(audio)

'the stale smell of old beer lingers'

만약 duration을 두 번 사용한다면, 두 번째에서는 첫 번째의 첫 4초 이후의 4초를 record함

In [10]:
with harvard as source:
    audio1 = r.record(source, duration=4)
    audio2 = r.record(source, duration=4)

r.recognize_google(audio1)

'the stale smell of old beer lingers'

In [11]:
r.recognize_google(audio2)

'it takes heat to bring out the odor a cold dip'

2. Offset은 첫 4초가 아닌 그 이후부터 record함

In [56]:
with harvard as source:
    audio = r.record(source, offset=4, duration=3)

r.recognize_google(audio)

'it takes heat to bring out the odor'

그런데 음성파일의 구조를 모른다면, 발화 내용이 잘려 제대로 인식되지 않을 수 있으니 주의해야 함

* 음성 인식 속 Noise

음성파일 속 Noise는 adjust_for_ambient_noise()로 보정될 수 있음

In [19]:
jackhammer = sr.AudioFile('jackhammer.wav')
with jackhammer as source:
    audio = r.record(source)
r.recognize_google(audio)

#실제 문장은'the stale smell of old beer lingers'이지만 노이즈로 인하여 ranscription이 다름

'the stale smell of old gear vendors'

In [21]:
#Recognizer 클래스의 adjust_for_ambient_noise() 메서드를 사용
with jackhammer as source:
    r.adjust_for_ambient_noise(source)
    audio = r.record(source)
    
r.recognize_google(audio)

'still smell like old beer drinkers'

첫 1초는 보정을 위한 구간으로 음성이 들어가면 안 되는데, Duration으로 이 구간을 조절할 수 있음

In [22]:
with jackhammer as source:
    r.adjust_for_ambient_noise(source, duration=0.5)
    audio = r.record(source)

r.recognize_google(audio)

'the stale smell of old gear vendors'

Noisy file의 경우 실제 API response를 보는 것이 도움이 되기도 함

In [23]:
#recognize_google()메서드를 True로 놓아 most likely transcription을 return함
r.recognize_google(audio, show_all=True)

{'alternative': [{'transcript': 'the stale smell of old gear vendors'},
  {'transcript': 'does still smell old gear vendors'},
  {'transcript': 'the smell smell of old gear vendors'},
  {'transcript': 'does still smell of old gear vendors'},
  {'transcript': 'the snail smell of old gear vendors'},
  {'transcript': 'the snail smelly old gear vendors'},
  {'transcript': 'does still smell olleh gear vendors'},
  {'transcript': 'does still smell bedgear vendors'}],
 'final': True}

# 3. Microphones 사용하기

마이크로 발화한 뒤 이를 텍스트로 바꿔봄

In [24]:
import speech_recognition as sr
r = sr.Recognizer() #Recognizer 클래스의 인스턴스를 생성함

In [33]:
mic = sr.Microphone() #디폴트 마이크를 사용

In [26]:
sr.Microphone.list_microphone_names() 
#마이크 리스트를 뽑아 Microphone의 device_index로 이 중 선택하는 것도 가능

['Microsoft Sound Mapper - Input',
 '마이크 배열(Realtek(R) Audio)',
 'Microsoft Sound Mapper - Output',
 '스피커(Realtek(R) Audio)',
 '주 사운드 캡처 드라이버',
 '마이크 배열(Realtek(R) Audio)',
 '주 사운드 드라이버',
 '스피커(Realtek(R) Audio)',
 '스피커(Realtek(R) Audio)',
 '마이크 배열(Realtek(R) Audio)',
 'Speakers (Realtek HD Audio output)',
 'Headphones (Realtek HD Audio 2nd output)',
 '스테레오 믹스 (Realtek HD Audio Stereo input)',
 '마이크 배열 (Realtek HD Audio Mic input)',
 '헤드폰 ()',
 '머리에 거는 수화기 (@System32\\drivers\\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0\r\n;(QCY-T2C))',
 '머리에 거는 수화기 (@System32\\drivers\\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0\r\n;(QCY-T2C))',
 '헤드폰 ()',
 '머리에 거는 수화기 (@System32\\drivers\\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0\r\n;(QCY-T2S_R))',
 '머리에 거는 수화기 (@System32\\drivers\\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0\r\n;(QCY-T2S_R))',
 '머리에 거는 수화기 (@System32\\drivers\\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0\r\n;(정의 AirPods))',
 '머리에 거는 수화기 (@System32\\drivers\\bthhfenum.sys,#2;%1 Hands-Free AG A

listen()을 메서드를 사용하여 Microphone Input을 캡처함

In [42]:
with mic as source:
    print("Say something") #녹음 시작과 끝에 구문을 출력하여 녹음을 더 쉽게 하도록 함
    audio = r.listen(source)
    print("Time over, Thanks")

Say something
Time over, Thanks


recognize_google()을 사용해 음성 파일을 텍스트로 변환함

In [43]:
r.recognize_google(audio)

'Hello nice to meet you how are you doing today'

* 주변 소음 조절하기

노이즈 파일과 같이 adjust_for_ambiet_noise() 메서드를 추가함

In [52]:
with mic as source:
    r.adjust_for_ambient_noise(source) 
    print("Say something")
    audio = r.listen(source)
    print("Time over, Thanks")
    
#음정 분석을 위해 첫 1초는 기다려야 함
#마찬가지로 duration으로 기다리는 시간을 조절할 수 있음

Say something
Time over, Thanks


In [53]:
r.recognize_google(audio)

'hello this is Harry Potter'